In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

def read_data_from_csv():
    hotels=pd.read_csv('zomato.csv')
    return hotels


def remove_unwanted_columns():
    hotels = read_data_from_csv() # assuming this function returns a DataFrame object
    unwanted_columns = ['address','phone'] # specify column names to be removed
    hotels = hotels.drop(columns=unwanted_columns) # remove unwanted columns
    return hotels # return updated DataFrame object

#task2: rename columns,  only these columns are allowed in the dataset
def rename_columns():
    hotels = remove_unwanted_columns() # assuming this function returns a DataFrame object
    column_mapping = {'name': 'name', 'rate': 'rating','approx_cost(for two people)':'approx_cost','listed_in(type)':'type'} # specify column name changes
    hotels = hotels.rename(columns=column_mapping) # rename columns
    return hotels # return updated DataFrame object
        

#task3: handle  null values of each column
def null_value_check():
    # call rename_columns() function to get dataframe
    hotels = rename_columns()
    
    # deleting null values of name column
    hotels = hotels.dropna(subset=['name'])
    
    # handling null values of online_order
    hotels['online_order'].fillna(value=False, inplace=True)
    
    # handling null values of book_table
    hotels['book_table'].fillna(value=False, inplace=True)
    
    # handling null values of rating
    hotels['rating'].fillna(value=0, inplace=True)
    #hotels['rating'].fillna(value=hotels['rating'].mean(), inplace=True)
    
    # handling null values of votes
    hotels['votes'].fillna(value=0, inplace=True)
    
    # handling null values of location
    hotels['location'].fillna(value='NA', inplace=True)
    #hotels['location'].fillna(value='Unknown', inplace=True)
    hotels['rest_type'].fillna(value='NA', inplace=True)
    
    hotels['dish_liked'].fillna(value='NA', inplace=True)
    
    hotels['cuisines'].fillna(value='NA', inplace=True)
    
    hotels['approx_cost'].fillna(value=0, inplace=True)

    hotels['type'].fillna(value='NA', inplace=True)

    # return the cleaned DataFrame
    return hotels

#task4 #find duplicates in the dataset
def find_duplicates():
    # call null_value_check() function to get dataframe
    hotels = null_value_check()
    
    # drop duplicates, keeping the first occurrence
    hotels.drop_duplicates(keep='first', inplace=True)
    
    # return the cleaned DataFrame
    return hotels



#task5 removing irrelevant text from all the columns
def removing_irrelevant_text():
    # call find_duplicates() function to get dataframe
    hotels = find_duplicates()

    # remove irrelevant text from all columns using string manipulation methods
    hotels = hotels[hotels['name'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['online_order'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['book_table'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['rating'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['votes'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['location'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['rest_type'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['dish_liked'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['cuisines'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['approx_cost'].str.contains('RATED|Rated') == False]
    hotels = hotels[hotels['type'].str.contains('RATED|Rated') == False]

    return hotels


#task6: check for unique values in each column and handle the irrelevant values
def check_for_unique_values():
    # call removing_irrelevant_text() function to get dataframe
    hotels = removing_irrelevant_text()
    hotels['online_order'] = hotels['online_order'].apply(lambda x: 'No' if x not in ['Yes', 'No'] else x)
    hotels = hotels[~hotels['rating'].str.contains('[a-zS]')]
    hotels['rating'] = hotels['rating'].replace(['-', 'NEW'], '0') 
    hotels['rating'] = hotels['rating'].str.replace('/5', '')
    def convert_to_int(x):
        if x == '0':
            return 0
        else:
            return x
    hotels['rating'] = hotels['rating'].apply(convert_to_int)
    return hotels
    
    

import re
#task7: remove the unknown character from the dataset and export it to "zomatocleaned.csv"
def remove_the_unknown_character():
    #DO NOT REMOVE FOLLOWING LINE
    #call check_for_unique_values() function to get dataframe
    dataframe = check_for_unique_values()
    dataframe['name'] = dataframe['name'].str.replace('[Ãx][^A-Za-z]+','',regex=True)
    
    dataframe.to_csv('zomatocleaned.csv')
    return dataframe



    #check if mysql table is created using "zomatocleaned.csv"
#Use this final dataset and upload it on the provided database for performing analysis in  MySQL
#To Run this task first Run the appliation for Terminal to create table named 'Zomato' and then run test.
def start():
    remove_the_unknown_character()

def task_runner():
    start()

**Task1 -For a high-level overview of the hotels, provide us the top 5 most voted hotels in the delivery category.**

**Task2 - The rating of a hotel is a key identifier in determining a restaurant’s performance. Hence for a particular location called Banashankari find out the top 5 highly rated hotels in the delivery category.**

**Task3 - compare the ratings of the cheapest and most expensive hotels in Indiranagar**

select
(select rating from zomato where location="Indiranagar" order by approx_cost limit 1) as rating1,
(select rating from zomato where location="Indiranagar" order by approx_cost desc limit 1) as rating2
from zomato limit 1;

**Task4 - Online ordering of food has exponentially increased over time. Compare the total votes of restaurants that provide online ordering services and those who don’t provide online ordering service**

**Task5 - Number of votes defines how much the customers are involved with the service provided by the restaurants For each Restaurant type, find out the number of restaurants, total votes, and average rating. Display the data with the highest votes on the top( if the first row of output is NA display the remaining rows).**

**Task6 - What is the most liked dish of the most-voted restaurant on Zomato(as the restaurant has a tie-up with Zomato, the restaurant compulsorily provides online ordering and delivery facilities.**

In [ ]:
SELECT dish, SUM(votes) AS total_votes
FROM zomato
WHERE name = (SELECT name FROM zomato ORDER BY votes DESC LIMIT 1)
GROUP BY dish
ORDER BY total_votes DESC
LIMIT 1;

**Task7 - To increase the maximum profit, Zomato is in need to expand its business. For doing so Zomato wants the list of the top 15 restaurants which have min 150 votes, have a rating greater than 3, and is currently not providing online ordering. Display the restaurants with highest votes on the top.**

In [ ]:
SELECT name, votes, rating, online_order
FROM zomato
WHERE online_order = 'No' AND votes >= 150 AND rating > 3
ORDER BY votes DESC
LIMIT 15;




In [ ]:
Expected Output: 
    Total Records Fetched: 6
    First 10 records of your query
    Headers: 
    name, rating, votes, online_order, 
    
    Values:
    Banashankari Donne Biriyani, 4.0, 1047, No, 
    Kabab Magic, 4.0, 808, No, 
    Caf-Eleven, 4.0, 424, No, 
    Roving Feast, 4.3, 345, No, 
    Chinese Kitchen, 4.2, 244, No, 
    Grand Village, 3.8, 166, No,